In [25]:
import qualified Data.Map as Map
import qualified Data.Set as Set
import Control.Monad.Writer
import Control.Monad.Identity
import Control.Monad.State
import Control.Monad.Reader
import Control.Monad.Trans.Maybe

In [2]:
type Vertex = Int
type Cost = Int
data Edge = Edge { vertex:: Vertex, cost:: Cost } deriving (Eq, Show, Ord)

In [3]:
newtype Graph = Graph { adjacencyList :: Map.Map Vertex [Edge] } deriving (Eq, Show)

In [4]:
constructGraph :: [(Vertex, Edge)] -> Graph
constructGraph edges = Graph $ foldr (\e g -> Map.insertWith (++) (fst e) [(snd e)] g) Map.empty edges

In [5]:
graph = constructGraph [(0, Edge 1 2), (1, Edge 0 2), (1, Edge 2 2), (2, Edge 1 2), (2, Edge 3 3),
    (3, Edge 2 3), (1, Edge 3 2), (3, Edge 1 2), (0, Edge 3 1), (3, Edge 0 1), (0, Edge 4 1), (4, Edge 0 1)]

<h4>DFS and BFS</h4>

In [6]:
getNeighbors :: Vertex -> Graph -> [Vertex]
getNeighbors v g = fmap vertex (Map.findWithDefault [] v (adjacencyList g))

filterVisitedNeighbors :: [Vertex] -> Set.Set Vertex -> [Vertex]
filterVisitedNeighbors listVertices set = filter (\x -> not (Set.member x set)) listVertices

In [8]:
bfs :: Vertex -> Graph -> [[Vertex]]
bfs s g = go [s] [] g (Set.singleton s) [[s]]
    where go :: [Vertex] -> [Vertex] -> Graph -> Set.Set Vertex -> [[Vertex]] -> [[Vertex]]
          go [] [] g visited result = result
          go [] nextLevel g visited result = go nextLevel [] g visited (result ++ [nextLevel])
          go (x:xs) nextLevel g visited result = 
              go xs (nextLevel ++ (unvisitedNeighbors x visited)) g (Set.union visited (Set.fromList $ unvisitedNeighbors x visited)) result
          unvisitedNeighbors x visited = filterVisitedNeighbors (getNeighbors x g) visited

In [9]:
bfsWithLogging :: Vertex -> Graph -> WriterT [String] Identity [[Vertex]]
bfsWithLogging s g = go [s] [] g (Set.singleton s) [[s]]
    where go :: [Vertex] -> [Vertex] -> Graph -> Set.Set Vertex -> [[Vertex]] -> WriterT [String] Identity [[Vertex]]
          go [] [] g visited result = return result
          go [] nextLevel g visited result = go nextLevel [] g visited (result ++ [nextLevel])
          go (x:xs) nextLevel g visited result = do
              tell ["Visiting :" ++ show x]
              go xs (nextLevel ++ (unvisitedNeighbors x visited)) g (Set.union visited (Set.fromList $ unvisitedNeighbors x visited)) result
          unvisitedNeighbors x visited = filterVisitedNeighbors (getNeighbors x g) visited

In [10]:
runIdentity . runWriterT $ bfsWithLogging 0 graph

([[0],[1,3,4],[2]],["Visiting :0","Visiting :1","Visiting :3","Visiting :4","Visiting :2"])

In [40]:
type Env = (Graph, Set.Set Vertex)
type DFSState = ([Vertex], Set.Set Vertex)
type Stack = [Vertex]

type DFSResult = ReaderT Env (MaybeT (StateT Vertex Identity)) DFSState

In [49]:
dfsNextVisit :: Stack -> DFSState -> Maybe ([Vertex], DFSState)
dfsNextVisit [] _ = Nothing
dfsNextVisit (x:xs) (g, visited) = if not (Set.member x visited) 
                                     then Just (x, (xs, Set.insert x visited))
                                   else dfsNextVisit xs (g, visited)                            

In [51]:
dfs :: Vertex -> Graph -> [Vertex]
dfs s g = go [s] g (Set.empty) []
    where go :: Stack -> Graph -> Set.Set Vertex -> [Vertex] -> [Vertex]
          go [] _ _ dfsTrail = dfsTrail
          go (x:xs) g visited dfsTrail = if not (Set.member x visited)
                                           then go ((getNeighbors x g) ++ xs) g (Set.insert x visited) (dfsTrail ++ [x])
                                         else go xs g visited dfsTrail

In [52]:
dfs 0 graph

[0,1,2,3,4]